In [1]:
import pandas as pd
import numpy as np
import os
import requests
from io import BytesIO
import json
import yadisk
from datetime import datetime, date, timedelta
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import config
from yandex_disk_func import *
from parse_functions import *
from google_connector import *
from date_time_func import *

# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'
# забираем токен для подключения к гугл
service_key = config.service
google_sheet_link = 'https://docs.google.com/spreadsheets/d/14sD1DTv6CfcsrVGnE0Ujkrc_EsGzVl9n5aiXKWp2hNM/edit?usp=sharing'#'https://docs.google.com/spreadsheets/d/1eakG0BJF1eSyi0hyGUpdw166TH_C9h_2zjnyrsMeejk/edit?usp=sharing' #'https://docs.google.com/spreadsheets/d/11hF_txYEbKrQImRQ0x3Fia3PyJvDAqtpGI25idV7oa4/edit?usp=sharing'
gmail = config.gmail
sheet_name = 'prog'

# обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета
public_key = config.public_key 

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
main_dict = {}

In [4]:
get_data_from_ya_folder(yandex_folders, main_dict, flag='prog')                     


/01_yandex
/02_prog
firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1) (1)
    mediaplan 1000к
    mediaplan 800к
    mediaplan 600к
hybrid_игроник_национальная лотерея_ко-промо принцесса нури_klientskiy_plan_20241115 (1)
    медиаплан 2024-2025
mobidriven_игроник_национальная лотерия_27.08 (1)
    мп
/03_split


In [5]:
df = pd.concat(main_dict, ignore_index=True)
df = df.fillna('')

df['views'] = df['views'].astype('float').round(0)
int_columns = ['reach', 'impressions', 'clicks', 'views']
df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))

In [6]:
float_columns = ['unit price', 'budget_without_nds', 'budget_nds', 'vtr, %']
df[float_columns] = df[float_columns].apply(lambda x: x.astype('float')).round(2)

In [7]:
df.columns

Index(['supplier', 'report_name', 'sheet_name', 'brand', 'period', 'source',
       'site/ssp', 'placement', 'targeting', 'geo', 'soc_dem',
       'ad copy format', 'rotation type', 'unit price', 'frequency', 'reach',
       'impressions', 'clicks', 'budget_without_nds', 'budget_nds', 'views',
       'vtr, %'],
      dtype='object')

In [8]:
# download_columns_list = ['supplier', 'report_name', 'sheet_name', 'brand', 'geo', 'soc_dem', 'ad copy format', 'rotation type', 'frequency', 'reach',
# 'impressions', 'clicks', 'budget_without_nds', 'budget_nds', 'views', 'vtr, %']

In [8]:
download_columns_list = df.columns

In [9]:
final_df = df[download_columns_list]

In [10]:
final_df

,supplier,report_name,sheet_name,brand,period,source,site/ssp,placement,targeting,geo,soc_dem,ad copy format,rotation type,unit price,frequency,reach,impressions,clicks,budget_without_nds,budget_nds,views,"vtr, %"
0,firstdata,firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1) (1),mediaplan 1000к,Национальная лотерея // Принцесса Нури,6 months,Платформа First Data,,,"ГЕО: РФ\nЦА: Ж, 35-45. Доход В, ВС. Покупатели чая и чайной продукции: Принцесса Нури, Азерчай, Ява, Riston, Майский, Ахмад, Dilmah, Ява, Милфорд, Липтон и др. Покупатели кондитерских изделий: безе, варенье, джем, повидло, грильяж, желе, зефир, пастила, конфеты, ирис, халва, помадка, муссы, шоколад, торты и др\nИсточники: ОФД",рф,"ж, 35-45. доход в, вс.",Баннеры / Универсальные Баннеры,CPM,240.0,12,2083333,25000000,50000,6000000.0,7200000.0,0,0.0
1,firstdata,firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1) (1),mediaplan 800к,Национальная лотерея // Принцесса Нури,6 months,Платформа First Data,,,"ГЕО: РФ\nЦА: Ж, 35-45. Доход В, ВС. Покупатели чая и чайной продукции: Принцесса Нури, Азерчай, Ява, Riston, Майский, Ахмад, Dilmah, Ява, Милфорд, Липтон и др. Покупатели кондитерских изделий: безе, варенье, джем, повидло, грильяж, желе, зефир, пастила, конфеты, ирис, халва, помадка, муссы, шоколад, торты и др\nИсточники: ОФД",рф,"ж, 35-45. доход в, вс.",Баннеры / Универсальные Баннеры,CPM,240.0,12,1666666,20000000,40000,4800000.0,5760000.0,0,0.0
2,firstdata,firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1) (1),mediaplan 600к,Национальная лотерея // Принцесса Нури,6 months,Платформа First Data,,,"ГЕО: РФ\nЦА: Ж, 35-45. Доход В, ВС. Покупатели чая и чайной продукции: Принцесса Нури, Азерчай, Ява, Riston, Майский, Ахмад, Dilmah, Ява, Милфорд, Липтон и др. Покупатели кондитерских изделий: безе, варенье, джем, повидло, грильяж, желе, зефир, пастила, конфеты, ирис, халва, помадка, муссы, шоколад, торты и др\nИсточники: ОФД",рф,"ж, 35-45. доход в, вс.",Баннеры / Универсальные Баннеры,CPM,260.0,12,1158333,13900000,27800,3614000.0,4336800.0,0,0.0
3,hybrid,hybrid_игроник_национальная лотерея_ко-промо принцесса нури_klientskiy_plan_20241115 (1),медиаплан 2024-2025,Национальная лотерея,15.11.2024 – 31.01.2025,hybrid,,Desktop+Mobile (web + in-app)***,"Национальная лотерея\nКо-промо Принцесса Нури\n\n\nКонструктор аудитории: Женщины, 35-45 лет, доход средний/выше среднего\n\n1. Аудиторные сегменты+Кастомные сегменты Hybrid Segments\n2. Hybrid Semantica \n3. Hybrid Places+SuperGeo на посетителей точек продаж столото и продуктовых магазинов.\n4. Сбор аудитории с сайтов во вкладке ""Пример сайтов""",РФ,,"Banner, Native",CPC,53.0,3,3393000,11310000,22620,1198860.0,1438632.0,0,0.0
4,mobidriven,mobidriven_игроник_национальная лотерия_27.08 (1),мп,Национальная Лотерия,01.11.24-30.04.25,Mobidriven,,in app,Ж 35-45 BC. Семейные с детьми. Работают в офисах или на гос. службе. Интересуются скидками и акциями.,РФ,Ж 35-45 лет,Fullscreen,CPM,290.0,3,6000000,18000000,270000,5220000.0,6264000.0,0,0.0
5,mobidriven,mobidriven_игроник_национальная лотерия_27.08 (1),мп,Национальная Лотерия,01.11.24-30.04.25,Mobidriven,,in web,Ж 35-45 BC. Семейные с детьми. Работают в офисах или на гос. службе. Интересуются скидками и акциями.,РФ,Ж 35-45 лет,"Banners (300*250, 320*100, 320*50)",CPM,110.0,3,5000000,15000000,45000,1650000.0,1980000.0,0,0.0
6,mobidriven,mobidriven_игроник_национальная лотерия_27.08 (1),мп,Национальная Лотерия,01.11.24-30.04.25,Mobidriven,,in app,Ж 35-45 BC. Семейные с детьми. Работают в офисах или на гос. службе. Интересуются скидками и акциями.,РФ,Ж 35-45 лет,Fullscreen,CPM,319.0,3,12000000,36000000,540000,11484000.0,13780800.0,0,0.0
7,mobidriven,mobidriven_игроник_национальная лотерия_27.08 (1),мп,Национальная Лотерия,01.11.24-30.04.25,Mobidriven,,in web,Ж 35-45 BC. Семейные с детьми. Работают в офисах или на гос. службе. Интересуются скидками и акциями.,РФ,Ж 35-45 лет,"Banners (300*250, 320*100, 320*50)",C

In [11]:
# создаем подключение к Гуглу
client = create_connection(service_key)

Connection established successfully...


In [12]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [13]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, final_df)

DataFrame exported successfully...


In [15]:
# создаем кастомный календарь для медиаплана
sheet_name = 'weekYear'
calendar_df = get_mediaplan_calendar(cur_year='2024', start_day_num=0)

start: 2024-01-01, end: 2024-12-31


In [16]:
# создаем подключение к Гуглу
client = create_connection(service_key)

Connection established successfully...


In [17]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [18]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, calendar_df)

DataFrame exported successfully...
